In [1]:
# https://www.math.uwaterloo.ca/~hwolkowi/matrixcookbook.pdf
# !pip install ase
!conda install nglview -c conda-forge -y
!conda install -c conda-forge kimpy openkim-models -y
# !conda install matplotlib -y
# !pip install nglview scikit-learn scipy ipympl ipywidgets matplotlib
# !conda install scipy -y

# !conda install tqdm -y
# !pip install sgdml
# !pip install sgdml[ase]
# !pip install sgdml[torch]
# !jupyter-nbextension enable nglview --py --sys-prefix
%load_ext autoreload
%autoreload 2
from utils import *
%matplotlib notebook

# Data-Driven Force Fields for Large Scale Molecular Dynamics Simulations of Halide Perovskites  

 
## Halide Perovskites (HaPs) 


## Theoretical Research of the Dynamics of Halide Perovskite 

Many of the properties of the HaPs are based on the unique structure and dynamics of the HaPs system. Also, numerous reports show that, even without defects, HaPs shows significant structural fluctuations.6 One of the main methods of investigating the properties of HaPs theoretically is molecular dynamics (MD). By employing the MD method one can obtain extensive dynamical information - including the actual location of the atoms, energy levels and the band structure, dipoles, and much more. 

## Molecular Dynamics (MD) 

Molecular Dynamics (MD) simulation is an algorithm for computing equilibrium and transport properties of many-body systems at a finite temperature. This method relies on the Newtonian equations of motion, updated each step numerically according to calculated forces. The applied forces of all the particles/atoms in the systems are updated by various possible methods, a few of which are discussed below.7  

### Classical Potential Molecular Dynamics 

Most common MD simulations are based on classical potentials. These potentials are usually based on principles of quantum mechanics but only in the form of fitted parameters, while the implementation of the equations is classical – atoms are regarded as points in space. The potential parameters need to be determined empirically for each set of materials. They are therefore limited in accuracy and types of systems.  

### First-Principles Molecular Dynamics 

First-principles molecular dynamics are dynamical simulations, in which forces on the atoms are calculated using first-principles methods. By far the most common first-principles method is density functional theory (DFT). First-principles calculations are considered more accurate than classical potentials and inherently take into account the quantum nature of the interaction of the atoms and the electron cloud. Their critical limitation, however, is the computational cost of a meaningful simulation. While it is possible to obtain many useful insights from those simulations, for HaPs, which are highly dynamical materials, these limitations are hard to surpass.  

## Molecular Dynamics of Halide Perovskite 

HaPs are comprised of at least three or more types of atoms and have many anharmonic features. This makes the usage of classical potential MD simulation very difficult for those materials. The only current possible option for dynamical research is of use first-principles MD, which limit the scale of the simulations. 

## Data-Driven Molecular Dynamics  

Until recently, constructing MD potentials was based on benchmark data and a small set of first-principles calculations, used to fit a classical MD potential. This process is highly specialized and involved, and required much understanding of the modeled system.  

The ever-increasing processing power and the rise of shared experimental and computational results data repositories provide the opportunity to easily obtaining a vast amount of information and data for each desired system. This change of setting allows the emergence of data-driven MD potentials. Those potentials are based on statistical learning, a branch of mathematics looking to extract and predict new properties from a given dataset.8  These approaches rely on much larger amounts of data compares to the classical approach. However, obtaining the potential is more general and can be easily applied to different systems. 

The two most common approaches are the artificial neural network (ANN) and the Gaussian process (GP). The methods are mathematical, but can be enhanced by using physical insights.9,10 The ANN is considered by many as a "black box" because the model has low interpretability but, by definition, can approximate any continuous function to a desired accuracy with enough data and a large enough network.11 GP's are somewhat limited in their prediction limits, but they are much easier to understand. Thus, it is possible to add deep physical insights that can drastically decrease the needed dataset size for the same accuracy.

## Gaussian Process Force-Field 
This work proposed to use the symmetrized gradient-domain machine learning (sGDML) method, a Gaussian process-based force field. The GP method is based on jointly gaussian distributions. Each configuration of the system is considered a single gaussian distribution coupled in an approximate manner to other configurations, by using a predefined "kernel function" which is considered a hyperparameter.

# Normal/Gaussian distribution
Normal distributions are important in statistics and are often used to represent real-valued random variables whose distributions are not known. Their importance is partly due to the [central limit theorem](https://en.wikipedia.org/wiki/Central_limit_theorem). It states that, under some conditions, the average of many samples (observations) of a random variable with finite mean and variance is itself a random variable—whose distribution converges to a normal distribution as the number of samples increases. 

based on: https://peterroelants.github.io/posts/multivariate-normal-primer/

## Univariate normal distribution

Univariate/single normal distribution is the simple case that the for given single value-scaler we have continuous probability distribution.
$$
\mathcal{N}(\mu, \sigma^2)
$$

$$
p(x \mid \mu, \sigma) = \frac{1}{\sqrt{2\pi\sigma^2}} \exp{ \left( -\frac{(x - \mu)^2}{2\sigma^2}\right)}
$$

In [2]:
def univariate_normal(x, mean, variance):
    """pdf of the univariate normal distribution."""
    return ((1. / np.sqrt(2 * np.pi * variance)) * 
            np.exp(-(x - mean)**2 / (2 * variance)))

In [3]:
univariate_plot(univariate_normal)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=0.0, description='mu', max=3.0, min=-3.0), FloatSlider(value=1.0, desc…

## Multivariate normal distribution
multivariate normal distribution, multivariate Gaussian distribution, or joint normal distribution is a generalization of the one-dimensional (univariate) normal distribution to higher dimensions.

$$
p(\mathbf{x} \mid \mathbf{\mu}, \Sigma) = \frac{1}{\sqrt{(2\pi)^d \lvert\Sigma\rvert}} \exp{ \left( -\frac{1}{2}(\mathbf{x} - \mathbf{\mu})^T \Sigma^{-1} (\mathbf{x} - \mathbf{\mu}) \right)}
$$

$$
\mathcal{N}(\mathbf{\mu}, \Sigma) \\
\mathbf{\mu} = \begin{pmatrix} \mu_1 \\ \mu_2\\ \dots\\ \mu_n\end{pmatrix}, 
\mathbf{\Sigma} = \begin{pmatrix}    \Sigma_{11} & \Sigma_{12} &\dots & \Sigma_{1n}
                                  \\ \Sigma_{21} & \Sigma_{22} & \dots & \Sigma_{2n}
                                  \\ \dots & \dots & \dots & \dots
                                  \\ \Sigma_{n1} & \dots & \dots &\Sigma_{nn}\end{pmatrix}, \mathbf{\Sigma} =\mathbf{\Sigma^\top}: \Sigma_{21}=\Sigma_{21}
$$

In [4]:
def multivariate_normal(x, d, mean, covariance):
    """pdf of the multivariate normal distribution."""
    x_m = x - mean
    return (1. / (np.sqrt((2 * np.pi)**d * np.linalg.det(covariance))) * 
            np.exp(-(np.linalg.solve(covariance, x_m).T.dot(x_m)) / 2))

## 2D Multivariate normal distribution

$$
\mathcal{N}\left(
\begin{bmatrix}
0 \\
1
\end{bmatrix}, 
\begin{bmatrix}
1 & C \\
C & 1
\end{bmatrix}\right)
$$

In [5]:
plt.close('all')

multivariate_plot(multivariate_normal, nb_of_x=100)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='C', max=0.99, step=0.01), O…

## Marginal and Conditional normal distributions

If both $\mathbf{y_1}$ and $\mathbf{y_2}$ are [jointly normal](https://en.wikipedia.org/wiki/Multivariate_normal_distribution#Joint_normality) the random vectors defined as:
$$
\begin{bmatrix}
\mathbf{y_1} \\
\mathbf{y_2} 
\end{bmatrix}
\sim
\mathcal{N}\left(
\begin{bmatrix}
\mu_{\mathbf{1}} \\
\mu_{\mathbf{2}}
\end{bmatrix},
\begin{bmatrix}
\Sigma_{11} & \Sigma_{12} \\ 
\Sigma_{21} & \Sigma_{22}
\end{bmatrix}
\right)
= \mathcal{N}(\mu, \Sigma)
, \qquad 
$$

The [conditional distribution](https://en.wikipedia.org/wiki/Conditional_probability_distribution) of $\mathbf{y_1}$ given $\mathbf{y_2}$ is defined as:

$$
p(\mathbf{y_1} \mid \mathbf{y_2}) = \mathcal{N}(\mu_{1|2}, \Sigma_{1|2})
$$

With:
$$\mu_{1|2} = \mu_1 -\Sigma_{12}\Sigma_{22}^{-1}(x_2-\mu_2) \;
\Sigma_{1|2} = (\Sigma/\Sigma_{22})=\Sigma_{11}-\Sigma_{12}\Sigma_{22}^{-1}\Sigma_{21}$$



For more details, see [Proof 1](#Proof_1)



In [6]:
plt.close('all')
condition_plot(nb_of_x=100)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='x', max=1.0, min=-1.0), Flo…

# Gaussian Process 
Gaussian process (GP) Is a method predicting $y^*$ for a given $x^*$ and getting: $y_i=f(x_i )$. 
GP assumes that $p(f(x_1),...,f(x_N  ))$ is a jointly Gaussian, i.e., the probability of getting the full random vector (the known $y$ points and the the new $y^{*}$ points) is defined as a multidimensional gaussian with $\mu$ and $\Sigma(x)$, where $\Sigma(x)$ is calculated as $\Sigma_{ij}=k(x_i,x_j)$, and $k$ is a kernel function.


$$
\begin{pmatrix}
\mathbf{y} \\ 
y^*
\end{pmatrix}=
\begin{pmatrix}
\mathbf{f} \\ 
f^*
\end{pmatrix}~
\mathcal{N}
\begin{pmatrix}
\begin{pmatrix}
\mathbf{\mu(x, y)} \\ 
\mu^*(x^*)
\end{pmatrix},
\begin{pmatrix}
\mathbf{K(x,x)} & \mathbf{k(x^*,x)^*}\\ 
{\mathbf{k(x^*, x)^{*}}}^{\top} & {k(x^*,x^*)^{**}}
\end{pmatrix}
\end{pmatrix}
$$
<br/>
$\mathbf{f}$ - Vector of all observed $y_i$ values, $f=y_i=f(x_i)$.<br/>
$f^*$ - Prediction function for point $x^*$, $f^*=y^*=f(x^*)$.<br/>
$\mathbf{\mu}$ - Vector of all observed mean values.<br/>
$\mathbf{\mu^*}$- Mean values for the prediction for x^*. <br/>
$\mathbf{K}$ - Covariance matrix of all observed points<br/>
$\mathbf{k^*}$ - Kernel vector $k^*=k(x^*,x_i )$<br/>
${k^{**}}$ - Self kernel $k^{**}=k(x^*,x^* )$<br/>


## Applying the Conditional 
$$f(x^*)=f(x^*|\mathbf{y},\mathbf{x})=\mu(x^*)^* +{{\mathbf{k}}^{*}}^{\top}  \mathbf{K}^{-1}(\mathbf{y}-\mu)$$ <br/>
For simplicity, we can define the mean values to be zero<br/>
$$f(x^*) ={\mathbf{k^*}}^{\top}\mathbf{K}^{-1}\mathbf{y}=\sum_{i=1}^{N}\alpha_ik(x_i, x^*)$$

### The Conditional on A Nosey Dataset
If we have a training dataset with noisy function values $\mathbf{y} = \mathbf{f} + \boldsymbol\epsilon$ where noise $\boldsymbol\epsilon \sim \mathcal{N}(\mathbf{0}, \sigma_y^2 \mathbf{I})$ is independently added to each observation then the predictive distribution is given by

$$
f(x^*) ={\mathbf{k^*}}^{\top}\mathbf{K_y}^{-1}\mathbf{y}=\sum_{i=1}^{N}\alpha_ik(x_i, x^*)
$$
where $\mathbf{K}_y = \mathbf{K} + \sigma_y^2\mathbf{I}$.

### Kernel Function
the is the kernel function which can be regarded as a "distance" function which defines how strongly the value $f(x_i)$ is coupled to point $f(x_j)$.

#### The Most Popular Kernal
In this part we will use the squared exponential kernel, also known as Gaussian kernel or RBF kernel. which is the go to kernel for starting the GP regression.
$$
\kappa(\mathbf{x}_i,\mathbf{x}_j) = \sigma_f^2 \exp\left(-\frac{1}{2l^2}
  (\mathbf{x}_i - \mathbf{x}_j)^T
  (\mathbf{x}_i - \mathbf{x}_j)\right)\tag{10}
$$

Based on:\
https://juanitorduz.github.io/gaussian_process_reg/ \
http://krasserm.github.io/2018/03/19/gaussian-processes/

In [7]:
def kernel(X1, X2, l=1.0, sigma_f=1.0):
    '''
    Isotropic squared exponential kernel. Computes
    a covariance matrix from points in X1 and X2.

    Args:
        X1: Array of m points (m x d).
        X2: Array of n points (n x d).

    Returns:
        Covariance matrix (m x n).
    '''
    sqdist = np.sum(X1 ** 2, 1).reshape(-1, 1) + np.sum(X2 ** 2, 1) - 2 * np.dot(X1, X2.T)
    return sigma_f ** 2 * np.exp(-0.5 / l ** 2 * sqdist)

In [8]:
plt.close('all')
n_sample = 50
n_predict = 10

x = plot_noise_sin(n_sample, plot_line=True, label='sample')
x_add = plot_noise_sin(n_predict, label='prediction')

x = np.concatenate([x, x_add], axis=0)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
plt.close('all')
heapmap_kernal(x.reshape([-1, 1]),  kernel)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

$$
f(x^*) ={\mathbf{k^*}}^{\top}\mathbf{K_y}^{-1}\mathbf{y}=\sum_{i=1}^{N}\alpha_ik(x_i, x^*)
$$

In [10]:
def posterior_predictive(X_s, X_train, Y_train, l=1.0, sigma_f=1.0, sigma_y=1e-8):
    '''
    Computes the suffifient statistics of the GP posterior predictive distribution
    from m training data X_train and Y_train and n new inputs X_s.

    Args:
        X_s: New input locations (n x d).
        X_train: Training locations (m x d).
        Y_train: Training targets (m x 1).
        l: Kernel length parameter.
        sigma_f: Kernel vertical variation parameter.
        sigma_y: Noise parameter.

    Returns:
        Posterior mean vector (n x d) and covariance matrix (n x n).
    '''
    K = kernel(X_train, X_train, l, sigma_f) + sigma_y ** 2 * np.eye(len(X_train))
    K_s = kernel(X_train, X_s, l, sigma_f)
    K_ss = kernel(X_s, X_s, l, sigma_f) 
    K_inv = inv(K)


    alphas = K_inv.dot(Y_train)
    mu_s = K_s.T.dot(alphas)


    cov_s = K_ss - K_s.T.dot(K_inv).dot(K_s)

    return mu_s, cov_s

In [11]:
plt.close()
noise = 0.1
x = [-3, -2 , -1,  1, 2 , 3, 4]
def f(x):
    func = np.exp(np.sin(x))
    return func
    
gaussian_process(x, f, noise, posterior_predictive, kernel)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
plt.close('all')
noise = 0.1
kernal = ConstantKernel(1.0) * RBF(length_scale=1.0)

gaussian_process_interactive(f, noise, kernal)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## GP Force Field 

A straightforward formulation of a vector-valued estimator
takes the form:
$$
\mathbf{\hat{f}}=\begin{bmatrix}\hat{f}_1(x), \dots, \hat{f}_N\end{bmatrix}^{T}
$$
$\mathbf{\hat{f}}: \mathbb{R}^N\rightarrow\mathbb{R}^N$ where each component: ${\hat{f}_i}: \mathbb{R}^N\rightarrow\mathbb{R}$

and it easy to applied it directly for prediction the forces on $N$ atoms system with $N$ 3D position vector and $N$ 3D forces vectors:
$\mathbf{\hat{f}}: \mathbb{R}^{3N}\rightarrow\mathbb{R}^{3N}$ where each component: ${\hat{f}_i}: \mathbb{R}^{3N}\rightarrow\mathbb{R}$

In [50]:
plt.close('all')
from ase_utils import *
"""Demonstrates molecular dynamics with constant temperature."""


a = 3.519
atoms_type = 'Ni'
atoms = Atoms([Atom(atoms_type, (0, 0, 0)), 
               Atom(atoms_type, (0, a/2, a/2)),
               Atom(atoms_type, (a/2, 0, a/2)),
               Atom(atoms_type, (a/2, a/2, 0))])
atoms.set_cell([a, a, a])
atoms.set_pbc(True)
atoms = atoms*2
atoms_copy = atoms.copy()
lat = np.array(atoms.get_cell())

ngl.view_ngl(atoms, w=500, h=500)

In [51]:
from ase.calculators.emt import EMT
from ase.calculators.kim.kim import KIM
T = 300  # Kelvin
# Describe the interatomic interactions with the Effective Medium Theory
# atoms.calc = EMT()
atoms.calc = KIM("EAM_Dynamo_MishinFarkasMehl_1999_Ni__MO_400591584784_005")


In [52]:
# A molecular dynamics object will operate on the atoms by moving them according to their forces - 
# it integrates Newton’s second law numerically

# Set the momenta corresponding T
MaxwellBoltzmannDistribution(atoms, T * 4 * units.kB)
# Two times room temperature simulation
dyn = Langevin(atoms, 1 * units.fs, 2*T * units.kB, 0.002)
dyn.attach(printenergy(atoms), interval=500)
# We also want to save the positions of all atoms after every 100th time step.
traj = Trajectory('reference.traj', 'w', atoms)
dyn.attach(traj.write, interval=1)

In [87]:
# Now run the dynamics
steps = 80000 - 1
dyn.run(steps)

Energy per atom: Epot = -4.354eV  Ekin = 0.094eV (T=724K)  Etot = -4.261eV, time elapsed: 2704.1
Energy per atom: Epot = -4.359eV  Ekin = 0.092eV (T=710K)  Etot = -4.267eV, time elapsed: 2705.2
Energy per atom: Epot = -4.347eV  Ekin = 0.079eV (T=614K)  Etot = -4.268eV, time elapsed: 2706.3
Energy per atom: Epot = -4.379eV  Ekin = 0.090eV (T=697K)  Etot = -4.289eV, time elapsed: 2707.3
Energy per atom: Epot = -4.363eV  Ekin = 0.076eV (T=590K)  Etot = -4.286eV, time elapsed: 2708.4
Energy per atom: Epot = -4.368eV  Ekin = 0.085eV (T=660K)  Etot = -4.283eV, time elapsed: 2709.5
Energy per atom: Epot = -4.379eV  Ekin = 0.092eV (T=715K)  Etot = -4.287eV, time elapsed: 2710.5
Energy per atom: Epot = -4.367eV  Ekin = 0.083eV (T=639K)  Etot = -4.284eV, time elapsed: 2711.6
Energy per atom: Epot = -4.378eV  Ekin = 0.103eV (T=797K)  Etot = -4.275eV, time elapsed: 2712.7
Energy per atom: Epot = -4.366eV  Ekin = 0.082eV (T=636K)  Etot = -4.284eV, time elapsed: 2713.8
Energy per atom: Epot = -4.382

Energy per atom: Epot = -4.364eV  Ekin = 0.061eV (T=469K)  Etot = -4.303eV, time elapsed: 2796.8
Energy per atom: Epot = -4.377eV  Ekin = 0.069eV (T=535K)  Etot = -4.308eV, time elapsed: 2797.9
Energy per atom: Epot = -4.377eV  Ekin = 0.069eV (T=532K)  Etot = -4.308eV, time elapsed: 2799.1
Energy per atom: Epot = -4.378eV  Ekin = 0.071eV (T=551K)  Etot = -4.307eV, time elapsed: 2800.5
Energy per atom: Epot = -4.386eV  Ekin = 0.072eV (T=557K)  Etot = -4.314eV, time elapsed: 2801.6
Energy per atom: Epot = -4.388eV  Ekin = 0.080eV (T=621K)  Etot = -4.308eV, time elapsed: 2802.7
Energy per atom: Epot = -4.384eV  Ekin = 0.083eV (T=645K)  Etot = -4.301eV, time elapsed: 2803.8
Energy per atom: Epot = -4.381eV  Ekin = 0.087eV (T=675K)  Etot = -4.293eV, time elapsed: 2804.9
Energy per atom: Epot = -4.371eV  Ekin = 0.074eV (T=571K)  Etot = -4.298eV, time elapsed: 2806.0
Energy per atom: Epot = -4.388eV  Ekin = 0.085eV (T=655K)  Etot = -4.304eV, time elapsed: 2807.1
Energy per atom: Epot = -4.394

True

In [54]:
traj = Trajectory('reference.traj', 'r', atoms)
ngl.view_ngl(traj, w=500, h=500)

## The Vector-Valued Estimator Problem
When mapping a scalar function $\mathbb{R}^{N}\rightarrow\mathbb{R}$ each entry in the kernel function correspond to one input vector ($x_i$). In the vector output case, we have multiple outputs for each input. Because of this limitation, we need to construct our kernel function carefully.

### The Navie Approch:
We can create an independent estimator for each value in the force vector.
$$
\mathbf{\hat{f}}=\begin{bmatrix}\hat{f}_1(x), \dots, \hat{f}_N\end{bmatrix}^{T}\\
$$
$$
\hat{f}_i(x) = \mathcal{GP}\begin{bmatrix}\mathbf{\mu(x)_i}, \mathbf{k(x,x^{'})_i}\end{bmatrix}\\
$$
$$
\mathbf{\hat{f}}=\begin{bmatrix}\hat{f}_1(x) \\ \dots \\ \hat{f}_N(x)\end{bmatrix}
=\begin{bmatrix}\mathcal{GP}\begin{bmatrix}\mathbf{\mu(x)_1}, \mathbf{k(x,x^{'})_1}\end{bmatrix} \\ 
\dots
\\ \mathcal{GP}\begin{bmatrix}\mathbf{\mu(x)_N}, \mathbf{k(x,x^{'})_N}\end{bmatrix}\end{bmatrix}
$$

In [74]:
traj = Trajectory('reference.traj', 'r', atoms)
start, every = 100, 10
(x, y), (x_test, y_test) = md_dataset_split(traj, start, every)
print(f'train points: {len(x)}')
noise = 0.2
steps = 20000

train points: 996


In [75]:
from sklearn.gaussian_process.kernels import ConstantKernel, RBF
kernal= ConstantKernel(1.0) * RBF(length_scale=1.0)
gpr = GaussianProcessRegressor(kernel=kernal, alpha=noise**2)
gpr.fit(x, y)
plot_gpr(gpr, x, y, x_test, y_test)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
import gc
gc.collect()
atoms = atoms_copy.copy()
atoms.calc = GPRCalculator(gpr)
MaxwellBoltzmannDistribution(atoms, T * 2 * units.kB)
dyn = Langevin(atoms, 1 * units.fs, T * units.kB, 0.002)
traj = Trajectory('moldyn_gdr.traj', 'w', atoms)
interval = 1
dyn.attach(printenergy_slim(steps, interval), interval=interval)
dyn.attach(traj.write, interval=1)
dyn.run(steps)
traj = Trajectory('moldyn_gdr.traj', 'r', atoms)
ngl.view_ngl(traj, w=500, h=500)


### Sme Tricks
##### change the kernal:
##### The Matérn kernel
It is generally assumed that overly smooth priors are detrimental to data efficiency, even if the prediction target is in fact indefinitely differentiable. For that reason, we will try to uses a Matérn kernel with restricted differentiability to construct the force field kernel function,
$$
k: C_{\mu=n+\frac{1}{2}}(d)=exp{-\frac{\sqrt{2\nu}d}{\sigma}}P_{n}(d)
$$
$$
P_n(d)=\sum_{k=0}^{n}{\frac{(n+k)!}{(2n)!}}\begin{pmatrix}n\\k\end{pmatrix}\begin{pmatrix}\frac{2\sqrt{2\nu}d}{\sigma}\end{pmatrix}^{n-k}
$$

In [76]:
from sklearn.gaussian_process.kernels import ConstantKernel, RBF, Matern
kernal= Matern(length_scale=1.0, nu=2.5)
gpr = GaussianProcessRegressor(kernel=kernal, alpha=noise**2)
gpr.fit(x, y)
plot_gpr(gpr, x, y, x_test, y_test)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
atoms = atoms_copy.copy()
atoms.calc = GPRCalculator(gpr)
MaxwellBoltzmannDistribution(atoms, T * 2 * units.kB)
dyn = Langevin(atoms, 1 * units.fs, T * units.kB, 0.002)
traj = Trajectory('moldyn_gdr.traj', 'w', atoms)
interval = 1
dyn.attach(printenergy_slim(steps, interval), interval=interval)
dyn.attach(traj.write, interval=1)
dyn.run(steps)
traj = Trajectory('moldyn_gdr.traj', 'r', atoms)
ngl.view_ngl(traj, w=500, h=500)

#### Normalize y
By normalizing the output value, we can "center" our output values around zero and $\sigma=1$ for all dimensions and reduce the overexpression of a certain axis.
$$
\hat{y}_{train} = \frac{y_{train}-E[y_{train}]}{\sqrt{Var[y_{train}]}}
$$

In [77]:
kernal= Matern(length_scale=1.0, nu=2.5)
gpr = GaussianProcessRegressor(kernel=kernal, alpha=noise**2, normalize_y=True)
gpr.fit(x, y)
plot_gpr(gpr, x, y, x_test, y_test)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [25]:
xatoms = atoms_copy.copy()
atoms.calc = GPRCalculator(gpr)
MaxwellBoltzmannDistribution(atoms, T * 2 * units.kB)
dyn = Langevin(atoms, 1 * units.fs, T * units.kB, 0.002)
steps = 30000
traj = Trajectory('moldyn_gdr.traj', 'w', atoms)
interval = 1
dyn.attach(printenergy_slim(steps, interval), interval=interval)
dyn.attach(traj.write, interval=1)
dyn.run(steps)
traj = Trajectory('moldyn_gdr.traj', 'r', atoms)
ngl.view_ngl(traj, w=500, h=500)

### Roto-translational invariance
Covariance functions remain valid under any transformation of their domain
) is again a kernel function. A rather trivial implication is that all
invariances of that input transformation are inherited, providing yet another opportunity
to characterize the properties of the predictor [92].

The so-called Coulomb matrix representation [7] goes one step further and represents
each pair of nuclei in terms their Coulomb interaction instead of a simple distance. The
Coulomb energy is the only nuclei-nuclei interaction term in the Hamiltonian and empirically a good starting point for inference about molecular properties [9]. We use a slight
variation of this descriptor for our purpose, whereby atoms of different type interact on a
normalized scale,
$$
D_{ij}=\begin{matrix}\lVert{r_i - r_j}\lVert & i > j \\ 0 & i \leq j \end{matrix} 
$$

In [78]:
d, d_test = x_to_d(x, lat), x_to_d(x_test,lat)
kernal= Matern(length_scale=1.0, nu=2.5)
gpr = GaussianProcessRegressor(kernel=kernal, alpha=noise**2, normalize_y=True)
gpr.fit(d, y)
plot_gpr(gpr, d, y, d_test, y_test)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [28]:
atoms = atoms_copy.copy()
atoms.calc = iGPRCalculator(gpr, lat)
MaxwellBoltzmannDistribution(atoms, T * 2 * units.kB)
dyn = Langevin(atoms, 1 * units.fs, T * units.kB, 0.002)
traj = Trajectory('moldyn_gdr.traj', 'w', atoms)
interval = 1
dyn.attach(printenergy_slim(steps, interval), interval=interval)
dyn.attach(traj.write, interval=10)
dyn.run(steps)
traj = Trajectory('moldyn_gdr.traj', 'r', atoms)
ngl.view_ngl(traj, w=500, h=500)

## Using Physical Insights to Build the Block Kernel
### Applying a Linear Operator
One way to construct the block kernel is by starting with a single-value estimator $\hat{f}$ and then applying a linear operator - $\hat{G}$ on it to obtain a vector estimator and obtain the following GP:
$$
\mathbf{\hat{g}(x)} ∼ \mathcal{GP}\begin{bmatrix}\mathbf{\hat{G}\mu(x)}, \hat{G}\mathbf{k(x,x^{'})}{\hat{G}'}^{T}\end{bmatrix}
$$

for A, B linear operators
$$
Cov[Ax,By]=ACov[x,y]B^{T}\\
E[Ax] = AE[x]
$$

### Conservative Force Field
In our case we are looking for crating an estimator for the force field. One of the most fundamental constrain on any force field is the preservation of energy i.e. the force field have to be a conservative force field. And in mathematical term we require that the curl vanishes  for every
input to the transformed energy model:
$$
\nabla\times \hat{G}\begin{bmatrix}\hat{f}\end{bmatrix} =\nabla\times \mathbf{f_F} = \mathbf{0}
$$
Not suprising the derivative operator $\nabla$ satisfied or, in the case of
energies and forces, the negative gradient operator
$$
\mathbf{\hat{f}_{F}(x)}=\hat{G}\begin{bmatrix}\hat{f}\end{bmatrix}=-\nabla\hat{f}
$$

And thank to notion that differentiation is a linear operator, the result is another GP with realizations $\mathbf{f_F}: \chi^{3N}\rightarrow \mathbb{R}^{3N}$
$$
\mathbf{\hat{f}_F(x)} ∼ \mathcal{GP}\begin{bmatrix}\mathbf{-\nabla_x\mu(x)}, \nabla_x\mathbf{k(x,x^{'})}{ \nabla_{x'}}^{T}\end{bmatrix}
$$

$$
\nabla\mathbf{k}{ \nabla}^{T}=Hess_x(k)
$$
$$
\begin{bmatrix}Hess_x(k)\end{bmatrix}_{ij}=\frac{\partial^2 k}{\partial x_i \partial x_j}
$$

## Gradient Domain Machine Learning (GDML)
This Hessian kernel gives rise to the following gradient domain machine
learning (GDML) force model as the posterior mean of the corresponding GP:<br>
he trained force field estimator collects the contributions of the
partial derivatives 3N of all training points M to compile the prediction.
It takes the form
$$
\mathbf{\hat{f}_F(x)}=\sum_i^M{\sum_j^{3N}{(\mathbf{\alpha_i})_j\frac{\partial}{\partial x_j}\nabla k(\mathbf{x}, \mathbf{x_i})}}
$$
    
Because the trained model is a (fixed) linear combination of kernel functions,
integration only affects the kernel function itself. The corresponding expression for
the energy predictor
$$
\mathbf{\hat{f}_E(x)}=\sum_i^M{\sum_j^{3N}{(\mathbf{\alpha_i})_j\frac{\partial}{\partial x_j} k(\mathbf{x}, \mathbf{x_i})}}
$$

For more details, see [Proof 2](#Proof_2)

### Roto-translational invariance
Covariance functions remain valid under any transformation of their domain $D : X → D,
i.e. k(D(x),D(x′)) = k_D(x,x′)$ is again a kernel function.
$$
D_{ij}=\begin{matrix}\lVert{r_i - r_j}\lVert & i > j \\ 0 & i \leq j \end{matrix} 
$$
And under the chain rule:
$$
\mathbf{k_F}=\nabla_x k_D \nabla_x^\top = \mathbf{J_D}^\top(\nabla_D k_D \nabla_D^\top)\mathbf{J_D}
$$
Where:
$$
\nabla_x = \begin{pmatrix}\frac{\partial}{\partial x_1} \\ \dots \\ \frac{\partial}{\partial x_{3N}}\end{pmatrix}=
\begin{pmatrix}\frac{\partial D_{11}}{\partial x_1}\frac{\partial}{\partial  D_{11}} +
                \frac{\partial D_{12}}{\partial x_1}\frac{\partial}{\partial  D_{12}} + 
                \dots
                \frac{\partial D_{1N}}{\partial x_1}\frac{\partial}{\partial  D_{1N}} +
                \frac{\partial D_{23}}{\partial x_1}\frac{\partial}{\partial  D_{23}} +
                \dots
                \frac{\partial D_{NN}}{\partial x_1}\frac{\partial}{\partial  D_{NN}}\\
                \dots \\ 
                \frac{\partial D_{11}}{\partial x_{3N}}\frac{\partial}{\partial  D_{11}} +
                \frac{\partial D_{12}}{\partial x_{3N}}\frac{\partial}{\partial  D_{12}} + 
                \dots
                \frac{\partial D_{1N}}{\partial x_{3N}}\frac{\partial}{\partial  D_{1N}} +
                \frac{\partial D_{23}}{\partial x_{3N}}\frac{\partial}{\partial  D_{23}} +
                \dots
                \frac{\partial D_{NN}}{\partial x_{3N}}\frac{\partial}{\partial  D_{NN}}\end{pmatrix}=
\begin{pmatrix}\frac{\partial D_{11}}{\partial x_1} &
               \frac{\partial D_{12}}{\partial x_1} &  
               \dots &
               \frac{\partial D_{1N}}{\partial x_1} &
               \frac{\partial D_{23}}{\partial x_1} & 
               \dots &
               \frac{\partial D_{NN}}{\partial x_1} \\
               \dots & \dots & \dots & \dots & \dots & \dots \\ 
               \frac{\partial D_{11}}{\partial x_{3N}} & 
               \frac{\partial D_{12}}{\partial x_{3N}} &  
               \dots &
               \frac{\partial D_{1N}}{\partial x_{3N}} &  
               \frac{\partial D_{23}}{\partial x_{3N}} & 
               \dots &
               \frac{\partial D_{NN}}{\partial x_{3N}} \end{pmatrix}
               \begin{pmatrix}\frac{\partial}{\partial D_{11}} \\ \dots \\ \frac{\partial}{\partial D_{NN}}\end{pmatrix}
               =\mathbf{J_D}^\top\nabla_D \\
               \\
\mathbf{J_D}=\begin{matrix}\frac{\mathbf{r_i} - \mathbf{r_j} }{\lVert{\mathbf{r_i}  - \mathbf{r_j}}\lVert^3} & i > j \\ 0 & i \leq j \end{matrix} 
$$

### The Matérn kernel
In the GDML model, a Matérn kernel functions is used
$$
k: C_{\nu=n+\frac{1}{2}}(d)=exp{-\frac{\sqrt{2\nu}d}{\sigma}}P_{n}(d)
$$
$$
P_n(d)=\sum_{k=0}^{n}{\frac{(n+k)!}{(2n)!}}\begin{pmatrix}n\\k\end{pmatrix}\begin{pmatrix}\frac{2\sqrt{2\nu}d}{\sigma}\end{pmatrix}^{n-k}
$$

### The Block kernel

$$
\mathbf{k_F(x, x^{'})}=\nabla k(x, x^{'})\nabla ^\top=\begin{bmatrix}\frac{\partial}{\partial x^{'}_1 }\nabla k(x,x^{'}),\dots,\frac{\partial}{\partial x^{'}_{3N} }\nabla k(x,x^{'})\end{bmatrix}
$$
$$
=\begin{pmatrix}5(\mathbf{x-x^{'}})(\mathbf{x-x^{'}})^\top-\mathbb{1}\sigma(\sigma +\sqrt{5}d))\end{pmatrix}\frac{5}{3\sigma^{4}}exp\begin{pmatrix}-\frac{\sqrt{5}d}{\sigma}\end{pmatrix}
$$
$$
\mathbf{k_F(x, x^{'})}\in \mathbb{R}^{3N \times 3N}
$$

$$
\mathbf{k_E(x, x^{'})}=k(x, x^{'})\nabla ^\top
=5(\mathbf{x-x^{'}})(\sigma+d)\frac{5}{3\sigma^{3}}exp\begin{pmatrix}-\frac{\sqrt{5}d}{\sigma}\end{pmatrix}
$$
$$
\mathbf{k_E(x, x^{'})}\in \mathbb{R}^{1 \times 3N}
$$


For more details, see [Derivation 3](#Derivation_3)


In [55]:
def block_kernal_F(x1, x2, sig=1):
    x1, x2 = x1.reshape([1, -1]), x2.reshape([1, -1])
    dim_x = x1.shape[1]
    x1, x2 = x1.reshape([dim_x, 1]), x2.reshape([dim_x, 1])
    
    d = np.linalg.norm(x1-x2)
    
    right_scalar = (5/(3*sig**4))*np.exp(-(np.sqrt(5)*d)/(sig))
    diagonal_sig = np.identity(dim_x)*sig*(sig+np.sqrt(d))
    
    x1x2 = x1-x2
    x1x2t = 5*np.matmul(x1x2, x1x2.T)
    
    return (x1x2t-diagonal_sig)*right_scalar

$$
\mathbf{k_F}=\nabla_x k_D \nabla_x^\top = \mathbf{J_D}^\top(\nabla_D k_D \nabla_D^\top)\mathbf{J_D}
$$

In [56]:
def Kf(x1, x2, sig):
    J_D1 = J_D(x1, lat)
    J_D2 = J_D(x2, lat)
    Kf = block_kernal_F(x_to_d(x1.reshape([1,-1]), lat), x_to_d(x2.reshape([1,-1]), lat) , sig)
    return J_D1.T.dot(Kf.dot(J_D2))

In [61]:
start, every = 0, 200
(x, y), (x_test, y_test) = md_dataset_split(traj, start, every)
print(f'train points: {len(x)}')
K = full_kernal(x, 1, Kf)


train points: 51


In [62]:
lam = 1e-10
K += lam*np.eye(K.shape[0])
K_inv = np.linalg.inv(K)
alpha = K_inv.dot(y.reshape([-1, 1]))

In [66]:
i = 10
x_s = x_test[i].reshape([1, -1])
k_s = k_star(x, x_s, 1, Kf)

y_s = y_test[i].flatten()
y_hat_s = k_s.T.dot(alpha).flatten()
error = np.linalg.norm(y_hat_s-y_s)/np.linalg.norm(y_s)
print(f'norm error of : {error*100} %')



norm error of : 99.48005652349381 %


In [88]:
from sgdml.cli import _print_dataset_properties, _print_model_properties, _print_task_properties

dataset_path = 'reference'
dataset = from_traj(f'{dataset_path}.traj', overwrite=True)
dataset_path = f'{dataset_path}.npz'
# test = np.load('ethanol_dft.npz')
_print_dataset_properties(dataset)

[INFO] Overwriting existing dataset file.
Writing dataset to 'reference.npz'...
Number geometries found: 99,999

[DONE]
Dataset properties
  Name:              reference (32 atoms)
  Theory:            unknown
  Size:              99,999 data points
  Lattice:           a     b     c    
                     7.038 0     0    
                     0     7.038 0    
                     0     0     7.038
    Lengths:         a = 7.038, b = 7.038, c = 7.038
    Angles [deg]:    alpha = 90, beta = 90, gamma = 90
  Energies [eV]:
    Range:           -1.42e+02 |--   3.77   --| -1.39e+02
    Mean:            -140.205
    Variance:        0.147
  Forces [eV/Ang]:
    Range:           -4.85 |--   9.43   --| 4.59     
    Mean:            0.000
    Variance:        0.695
  Fingerprint:       b'26b8e182d6646a27f86df737f73057ea'

Example geometry (no. 77,043, chosen randomly)
  Copy&paste the string below into Jmol (www.jmol.org), Avogadro (www.avogadro.cc), etc. to
  visualize a geometry from th

In [86]:
n_train = 500
n_valid = 10000
file = all_script(dataset_path, n_train, n_valid, use_cg=True, use_torch=True)
print(f'please run \"bash {file}\" from the terminal')

please run "bash run_me.sh" from the terminal


In [84]:
model_path = 'reference-unknown-train100-sym1.npz'
traj = Trajectory('reference.traj', 'r', atoms)
start, every = 100, 10
plot_gdml(model_path, traj, start, every)

(996, 96)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [58]:
from ase_utils import *
from sgdml.intf.ase_calc import SGDMLCalculator
from ase.optimize import QuasiNewton
import gc
model_path = 'reference-unknown-train200-sym1.npz'
calc = SGDMLCalculator(model_path)
atoms = atoms_copy.copy()
atoms.set_calculator(calc)
# set the momenta corresponding to T=300K
MaxwellBoltzmannDistribution(atoms, T * units.kB)
dyn = Langevin(atoms, 1 * units.fs, T * units.kB, 0.002, trajectory='moldyn_sgdml.traj')    
dyn.attach(printenergy(atoms), interval=500)

# now run the dynamics
printenergy(atoms)
dyn.run(steps)
gc.collect()


[WARN] Please remember to specify the proper conversion factors, if your model does not use
       'kcal/mol' and 'Ang' as units.


prepare


IndexError: index 4 is out of bounds for axis 0 with size 4

## Proof_1
$
\begin{split}
p(x_1, x_2) = exp
\left[
-\frac{1}{2}
\begin{pmatrix}
x_1 - \mu_1 \\ 
x_2 - \mu_2
\end{pmatrix}^T
\begin{pmatrix}
\Sigma_{11} & \Sigma_{12} \\ 
\Sigma_{21} & \Sigma_{22}
\end{pmatrix}^{-1}
\begin{pmatrix}
x_1 - \mu_1 \\ 
x_2 - \mu_2
\end{pmatrix}
\right]
\end{split}
$

By using thr following idendty:

$
\begin{split}
M^{-1}=
\begin{pmatrix}
A & B \\ 
C & D
\end{pmatrix}^{-1}=
\begin{pmatrix}
I & 0 \\ 
-C^{-1}C & I
\end{pmatrix}
\begin{pmatrix}
(M/D)^{-1} & 0 \\ 
0 & D^{-1}
\end{pmatrix}
\begin{pmatrix}
I & -BD^{-1} \\ 
0 & I
\end{pmatrix}
\end{split}
$

We obtainig the folowing:

$
\begin{pmatrix}
x_1 - \mu_1 \\ 
x_2 - \mu_2
\end{pmatrix}^T
\begin{pmatrix}
\Sigma_{11} & \Sigma_{12} \\ 
\Sigma_{21} & \Sigma_{22}
\end{pmatrix}^{-1}
\begin{pmatrix}
x_1 - \mu_1 \\ 
x_2 - \mu_2
\end{pmatrix}=
\begin{pmatrix}
x_1 - \mu_1 \\ 
x_2 - \mu_2
\end{pmatrix}^T
\begin{pmatrix}
I & 0 \\ 
-\Sigma_{22}^{-1} \Sigma_{21} & I
\end{pmatrix}
\begin{pmatrix}
(\Sigma/\Sigma_{22})^{-1} & 0 \\ 
0 & \Sigma_{22}^{-1}
\end{pmatrix}
\begin{pmatrix}
I & -\Sigma_{12}\Sigma_{22}^{-1} \\ 
0 & I
\end{pmatrix}
\begin{pmatrix}
x_1 - \mu_1 \\ 
x_2 - \mu_2
\end{pmatrix}
$

And finally:

$
(x_2 - \mu_1 -\Sigma_{12}\Sigma_{22}^{-1}(x_2-\mu_2))^T(\Sigma/\Sigma_{22})^{-1}(x_2 - \mu_1 -\Sigma_{12}\Sigma_{22}^{-1}(x_2-\mu_2)) + (x_2-\mu_2)^T\Sigma_22^{-1}(x_2-\mu_2)
$

And plugin back into the first Eq:

$
\begin{split}
p(x_1, x_2) = exp
\left[
-\frac{1}{2}
(x_2 - \mu_1 -\Sigma_{12}\Sigma_{22}^{-1}(x_2-\mu_2))^T(\Sigma/\Sigma_{22})^{-1}(x_2 - \mu_1 -\Sigma_{12}\Sigma_{22}^{-1}(x_2-\mu_2)) + (x_2-\mu_2)^T\Sigma_22^{-1}(x_2-\mu_2)
\right]
\end{split}
$
$
\begin{split}
p(x_1, x_2) = exp
\left[
-\frac{1}{2}
(x_2 - \mu_1 -\Sigma_{12}\Sigma_{22}^{-1}(x_2-\mu_2))^T(\Sigma/\Sigma_{22})^{-1}(x_2 - \mu_1 -\Sigma_{12}\Sigma_{22}^{-1}(x_2-\mu_2))
\right] \cdot
exp
\left[
-\frac{1}{2}
(x_2-\mu_2)^T\Sigma_22^{-1}(x_2-\mu_2)
\right]
\end{split}
$

and from the axiom of probability we obtain the follwoing equlaity:

$
\begin{split}
p(x_1, x_2) = p(x_1|x_2)p(x_2)= exp
\left[
-\frac{1}{2}
(x_2 - \mu_1 -\Sigma_{12}\Sigma_{22}^{-1}(x_2-\mu_2))^T(\Sigma/\Sigma_{22})^{-1}(x_2 - \mu_1 -\Sigma_{12}\Sigma_{22}^{-1}(x_2-\mu_2))
\right] \cdot
exp
\left[
-\frac{1}{2}
(x_2-\mu_2)^T\Sigma_22^{-1}(x_2-\mu_2)
\right] = 
exp
\left[
-\frac{1}{2}
(x_2 - \mu_1 -\Sigma_{12}\Sigma_{22}^{-1}(x_2-\mu_2))^T(\Sigma/\Sigma_{22})^{-1}(x_2 - \mu_1 -\Sigma_{12}\Sigma_{22}^{-1}(x_2-\mu_2))
\right] \cdot
p(x_2)
\end{split} \\ 
\Rightarrow p(x_1|x_2) = exp
\left[
-\frac{1}{2}
(x_2 - \mu_1 -\Sigma_{12}\Sigma_{22}^{-1}(x_2-\mu_2))^T(\Sigma/\Sigma_{22})^{-1}(x_2 - \mu_1 -\Sigma_{12}\Sigma_{22}^{-1}(x_2-\mu_2))
\right] \\
\mu_{1|2} = \mu_1 -\Sigma_{12}\Sigma_{22}^{-1}(x_2-\mu_2) \;
\Sigma_{1|2} = (\Sigma/\Sigma_{22})=\Sigma_{11}-\Sigma_{12}\Sigma_{22}^{-1}\Sigma_{21}
$

## Proof_2

### For scalar output
$$
\begin{bmatrix}f^*\end{bmatrix}^{1\times1} = \begin{bmatrix}{k^*}^T\end{bmatrix}^{1\times M}\begin{bmatrix}\begin{bmatrix}K^{-1}\end{bmatrix}^{M \times M}\begin{bmatrix}y\end{bmatrix}^{M\times 1}\end{bmatrix}^{M\times1}=\sum_{i}^{M}\alpha_i k(x_i,x^*)
$$

### For vector output
$$
\begin{bmatrix}\mathbf{f^*}\end{bmatrix}^{3N\times1} ={k^*}^{T}K^{-1}\mathbf{f}=\sum_i^M{\sum_j^{3N}{(\mathbf{\alpha_i})_j\frac{\partial}{\partial x_j}\nabla k(\mathbf{x}, \mathbf{x_i})}}
$$

$$
K=\nabla_{x} k(x,x') \nabla_{x^{'}}^\top = 
\begin{pmatrix}
\begin{pmatrix}\nabla_{x^{(1)}} k(x^{(1)},x^{(1)}) \nabla_{x^{(1)}}^\top\end{pmatrix} & \dots & \begin{pmatrix}\nabla_{x^{(M)}} k(x^{(M)},x^{(1)}) \nabla_{x^{(1)}}^\top\end{pmatrix} \\
\dots & \dots & \dots \\
\begin{pmatrix}\nabla_{x^{(M)}} k(x^{(M)},x^{(1)}) \nabla_{x^{(1)}}^\top\end{pmatrix} & \dots & \begin{pmatrix}\nabla_{x^{(M)}} k(x^{(M)},x^{(M)}) \nabla_{x^{(M)}}^\top\end{pmatrix}
\end{pmatrix}
\\
\nabla k(x^{(i)}, x^{(j)})\nabla ^\top=\begin{bmatrix}\frac{\partial}{\partial x^{(j)}_1 }\nabla k(x^{(i)},x^{(j)}),\dots,\frac{\partial}{\partial x^{(j)}_{3N} }\nabla k(x^{(i)},x^{(j)})\end{bmatrix}^{3N\times 3N}
=\begin{bmatrix}\frac{\partial}{\partial x^{(j)}_1 }\frac{\partial}{\partial x^{(i)}_{1}} k(x^{(i)},x^{(j)}) &\dots & \frac{\partial}{\partial x^{(j)}_{3N} }\frac{\partial}{\partial x^{(i)}_{1}} k(x^{(i)},x^{(j)}) \\
\dots & \dots & \dots \\
\frac{\partial}{\partial x^{(j)}_1}\frac{\partial}{\partial x^{(i)}_{3N}} k(x^{(i)},x^{(j)}) &\dots & \frac{\partial}{\partial x^{(j)}_{3N} }\frac{\partial}{\partial x^{(i)}_{3N}} k(x^{(i)},x^{(j)})
\end{bmatrix}^{3N\times 3N}
\\
\begin{bmatrix}K\end{bmatrix}^{M\cdot3N\times M\cdot3N} \;
\begin{bmatrix}\mathbf{f}\end{bmatrix}^{M\cdot3N\times 1}
\\
$$
$$
{k^*}^{T}
=\nabla_x k(x^{*}, x)\nabla_{x^{'}} ^\top 
=
\begin{bmatrix}
\begin{pmatrix}\nabla_{x^*} k(x^{*}, x^{(1)})\nabla_{x^{(1)}} ^\top\end{pmatrix} & 
\dots & 
\begin{pmatrix}\nabla k(x^{*}, x^{(M)})\nabla_{x^{(M)}} ^\top\end{pmatrix}
\end{bmatrix}^{3N\times M\cdot 3N}
= 
\begin{bmatrix}
\begin{pmatrix}\frac{\partial}{\partial x^{(1)}_1 }\nabla k(x^{*},x^{(1)}),\dots,\frac{\partial}{\partial x^{(1)}_{3N} }\nabla k(x^{*},x^{(1)})\end{pmatrix} & 
\dots & 
\begin{pmatrix}\frac{\partial}{\partial x^{(M)}_1 }\nabla k(x^{*},x^{(M)}),\dots,\frac{\partial}{\partial x^{(M)}_{3N} }\nabla k(x^{*},x^{(M)})\end{pmatrix}
\end{bmatrix}^{3N\times M\cdot 3N}
\\
\\
\begin{bmatrix}\mathbf{f^*}\end{bmatrix}^{3N\times1} =\begin{bmatrix}\begin{bmatrix}{k^*}^{T}\end{bmatrix}^{3N \times M \cdot 3N}\begin{bmatrix}\begin{bmatrix}K^{-1}\end{bmatrix}^{M\cdot 3N \times M\cdot 3N}\begin{bmatrix}\mathbf{f}\end{bmatrix}^{M\cdot 3N \times 1}\end{bmatrix}^{M \cdot 3N \times 1}\end{bmatrix}^{3N\times 1}
\\
\begin{bmatrix}\mathbf{f^*}\end{bmatrix}^{3N\times1} =
{k^*}^{T}
K^{-1}
\mathbf{f}=
\begin{pmatrix}
\begin{pmatrix}\frac{\partial}{\partial x^{(1)}_1 }\nabla k(x^{*},x^{(1)}),\dots,\frac{\partial}{\partial x^{(1)}_{3N} }\nabla k(x^{*},x^{(1)})\end{pmatrix} & 
\dots & 
\begin{pmatrix}\frac{\partial}{\partial x^{(M)}_1 }\nabla k(x^{*},x^{(M)}),\dots,\frac{\partial}{\partial x^{(M)}_{3N} }\nabla k(x^{*},x^{(M)})\end{pmatrix}
\end{pmatrix}
\underbrace{ K^{-1}\mathbf{f}}_{\alpha _{ij}}
$$

### Derivation_3 

### Matérn covariance derivatives
$$
k: C_{\nu=n+\frac{1}{2}}(d)=B(d)P_{n}(d)
\\
B(d) = exp{\begin{pmatrix}-\frac{\sqrt{2\nu}d}{\sigma}\end{pmatrix}}
\\
P_n(d)=\sum_{k=0}^{n}{\frac{(n+k)!}{(2n)!}}\begin{pmatrix}n\\k\end{pmatrix}\begin{pmatrix}\frac{2\sqrt{2\nu}d}{\sigma}\end{pmatrix}^{n-k}
$$

$$
\frac{\partial \kappa}{\partial x_i} = \frac{\partial P_n}{\partial x_i}B + \frac{\partial B}{\partial x_i}P_n
$$

$$
\frac{\partial P_n}{\partial x_i}=\sum_{k=0}^n{\frac{(n+k)!}{(2n)!}\begin{pmatrix}n\\k\end{pmatrix}\frac{(n-k)(x_i -x^{'}_i)}{d^2}\begin{pmatrix}\frac{2\sqrt{2\nu}d}{\sigma}\end{pmatrix}^{n-k}}\\
$$

$$
\frac{\partial B}{\partial x_i} =-\frac{\sqrt{2\nu}(x_i -x^{'}_i)}{\sigma d}exp{-\frac{\sqrt{2\nu}d}{\sigma}}
$$

$$
\frac{\partial^2 \kappa}{\partial x_i \partial x_j} = 
B\frac{\partial^2 P_n}{\partial x_i \partial x_j} +
\frac{\partial B}{\partial x_i}\frac{\partial P_n}{\partial x_j} +
\frac{\partial B}{\partial x_j}\frac{\partial P_n}{\partial x_i} +
P_n\frac{\partial^2 B}{\partial x_i \partial x_j} 
$$

$$
\begin{bmatrix}\frac{\partial^2 P_n}{\partial x_i \partial x_j}\end{bmatrix}_{i\ne j}=
\sum_{k=0}^{n}{\frac{(n+k)!}{(2n)!}\begin{pmatrix}n \\ k \end{pmatrix}
\frac{(n-k-2)(n-k)(x_i -x^{'}_i)(x_j - x^{'}_j)}{d^4}
\begin{pmatrix}\frac{2\sqrt{2\nu}d}{\sigma}\end{pmatrix}^{n-k}}
$$

$$
\begin{bmatrix}\frac{\partial^2 P_n}{\partial x_i \partial x_j}\end{bmatrix}_{i=j}=
\sum_{k=0}^{n}{\frac{(n+k)!}{(2n)!}\begin{pmatrix}n \\ k \end{pmatrix}
\frac{(n-k-2)(n-k)(x_i -x^{'}_i)^2}{d^4}
\begin{pmatrix}\frac{2\sqrt{2\nu}d}{\sigma}\end{pmatrix}^{n-k}} + \sum_{k=0}^n{\frac{(n+k)!}{(2n)!}\begin{pmatrix}n\\k\end{pmatrix}\frac{(n-k)}{d^2}\begin{pmatrix}\frac{2\sqrt{2\nu}d}{\sigma}\end{pmatrix}^{n-k}}\\
$$

$$
\begin{bmatrix}\frac{\partial^2 B}{\partial x_i \partial x_j}\end{bmatrix}_{i\ne j} =
\frac{\sqrt{2\nu}(x_i-x^{'})(x_j-x^{'}_j)(\sqrt{2\nu} d+\sigma)}{\sigma^2d^3}\exp{\begin{pmatrix}-\frac{\sqrt{2\nu}d}{\sigma}\end{pmatrix}}
$$

$$
\begin{bmatrix}\frac{\partial^2 B}{\partial x_i \partial x_j}\end{bmatrix}_{i = j} =
\frac{\sqrt{2\nu}(x_i-x^{'})(x_j-x^{'}_j)(\sqrt{2\nu} d+\sigma)}{\sigma^2d^3}\exp{\begin{pmatrix}-\frac{\sqrt{2\nu}d}{\sigma}\end{pmatrix}} - \frac{\sqrt{2\nu}(x_i -x^{'}_i)}{\sigma d}exp{\begin{pmatrix}-\frac{\sqrt{2\nu}d}{\sigma}\end{pmatrix}}
$$